In [3]:
%%writefile app.py
from flask import Flask, request, abort
import datetime
import csv
import os

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent

# =====================
# 設定
# =====================

CHANNEL_ACCESS_TOKEN = "xqHzNdKVt2TYRHsp88TQrMP8FBv8+n132rocBwALf6igVckAqX/fhww4uARTHiKXAjmxVlMGmOuqJMwX7dt4PABaBXrEGT4XXfDH8OerR8hoR7p/JdfMoBOIBBBxgVuwKomyDUzioKhpDikrtxC++wdB04t89/1O/w1cDnyilFU="
CHANNEL_SECRET = "63fa15201c651c5f5428a8a9d2087bda"

SAVE_PATH = "user_inputs.csv"

# ファイルが存在しない場合はヘッダ付きで作成
if not os.path.exists(SAVE_PATH):
    with open(SAVE_PATH, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "user_id", "message"])

# LINE設定
configuration = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# Flask
app = Flask(__name__)


# =====================
# Webhook endpoint
# =====================

@app.route("/callback", methods=["POST"])
def callback():

    signature = request.headers.get("X-Line-Signature")
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return "OK"


# =====================
# メッセージ受信処理
# =====================

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):

    user_id = event.source.user_id
    message = event.message.text

    # 現在時刻
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # CSVへ保存
    with open(SAVE_PATH, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        # writer.writerow([timestamp, user_id, message])
        writer.writerow([message])

    print(f"保存: {timestamp}, {user_id}, {message}")

    # 任意: 応答
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # line_bot_api.reply_message(
        #     ReplyMessageRequest(
        #         reply_token=event.reply_token,
        #         messages=[
        #             TextMessage(text="保存しました♡")
        #         ]
        #     )
        # )

Overwriting app.py


In [ ]:
!flask run --reload --port 8080 --without-threads
# ngrok http 8080

 * Debug mode: off
 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
保存: 2026-02-23 10:34:38, Uc838ce445b3c71787e5c3e5a70d2e289, あ
127.0.0.1 - - [23/Feb/2026 10:34:38] "POST /callback HTTP/1.1" 200 -
保存: 2026-02-23 10:34:48, Uc838ce445b3c71787e5c3e5a70d2e289, こんにちは
127.0.0.1 - - [23/Feb/2026 10:34:48] "POST /callback HTTP/1.1" 200 -
保存: 2026-02-23 10:35:06, Uc838ce445b3c71787e5c3e5a70d2e289, テスト10:35
127.0.0.1 - - [23/Feb/2026 10:35:06] "POST /callback HTTP/1.1" 200 -
保存: 2026-02-23 10:45:17, Uc838ce445b3c71787e5c3e5a70d2e289, テスト10:45
127.0.0.1 - - [23/Feb/2026 10:45:17] "POST /callback HTTP/1.1" 200 -
保存: 2026-02-23 10:45:31, Ud57ba83a26751be7e147c436dbb480b6, テスト
127.0.0.1 - - [23/Feb/2026 10:45:31] "POST /callback HTTP/1.1" 200 -
保存: 2026-02-23 10:45:58, Ud57ba83a26751be7e147c436dbb480b6, 私はどの大学が向いているでしょうか
127.0.0.1 - - [23/Feb/2026 10:45:58] "POST /callback HTTP/1.1" 200 -
保存: 2026-02-23 10:46:30, Ud57ba83a26751be7e147c436dbb480b6, PCのお